## For Colab only
We need to clone the repo here before we can use it

In [ ]:
!git init
!git remote add origin "https://github.com/karan-sarkar/VGGVox-PyTorch.git" 
!git pull origin master
!git branch --set-upstream-to=origin/master master

In [ ]:
!pip install -r requirements.txt

Download dataset from gdrive. The specified file is just a minimal copy of the dataset. You can upload a different file and change the ID to your file.

In [ ]:
!gdown --id '19q3G3XWKCkjwzdAWf8E-Ntx_hG-shRxo' -O data/wav.zip
!unzip ./data/wav.zip -d ./data/ && mv ./data/subset ./data/wav

In [ ]:
data_dir = "./data"

## Start

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import pandas as pd
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
sys.path.append(os.path.abspath('..'))
from train_fsl import Experiment

In [ ]:
try: data_dir
except: data_dir = "/Users/pi/repos/gatech/8803-LS/Project/dataset/voxceleb2"


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#@title Set training params { run: "auto" }
batch_size = 5 #@param {type:"integer"}
num_way = 5 #@param {type:"integer"}
backbone_arch = "resnet18" #@param ["resnet18", "resnet34", "audionet"]
num_workers = 0 #@param {type:"integer"}
num_shot = 5 #@param {type:"integer"}
num_query = 1 #@param {type:"integer"}
num_train_tasks = 20 #@param {type:"integer"}
num_val_tasks = 20 #@param {type:"integer"}
num_eval_tasks = 20 #@param {type:"integer"}
num_epochs = 20 #@param {type:"integer"}
augmentation = "none" #@param ["spec", "mixup", "none"]
fsl_arch = "relation-net" #@param ["relation-net", "proto-net"]


In [ ]:
N_WAY = 5 
experiment = Experiment(
    batch_size=batch_size, 
    num_workers=num_workers, 
    device=device,
    num_way=num_way,
    num_shot=num_shot,
    num_query=num_query,
    num_train_tasks=num_train_tasks,
    num_val_tasks=num_val_tasks,
    num_eval_tasks=num_eval_tasks,
    num_epochs=num_epochs,
    augmentation=augmentation,
    backbone_arch=backbone_arch,
    fsl_arch=fsl_arch,
    dir=data_dir,
)

In [ ]:
loaders = experiment.Dataloaders
batch = next(iter(loaders['train']))

In [ ]:
batch[0].shape

In [ ]:
def plot_freq(dataset):
    labels = np.array(dataset.labels)
    classes, counts = np.unique(labels, return_counts=True)
    print('The range of frequencies is :', min(counts), '->', max(counts))
    classes = list(map(str, classes))
    _ = plt.bar(classes, counts)
    plt.show()
print('Train')
plot_freq(loaders['train'].dataset)
print('Val')
[plot_freq(dl.dataset) for dl in loaders['val']]
print('Test')
[plot_freq(dl.dataset) for dl in loaders['test']]

In [ ]:
train_classes = loaders['train'].dataset.labels
val_classes = loaders['val'][0].dataset.labels
test_classes = loaders['test'][0].dataset.labels
print('The 3 datasets should not have any intersection')
print('Train-Val intersection:', set(train_classes).intersection(set(val_classes)))
print('Val-Test intersection:', set(test_classes).intersection(set(val_classes)))
print('Train-Test intersection:', set(train_classes).intersection(set(test_classes)))

In [ ]:
experiment.train()